In [1]:
import os
import sys

os.chdir("C:/dataanalytics/python")
os.curdir

#Configure the environment . Set this up to the directory where spark is installed
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = 'C:\\spark'
    
#create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. Please check your installation
#to make sure that these zip files actually exists. The names might change as
#versions change
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.6-src.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
 
#Initialize a spark context
from pyspark import SparkContext
from pyspark import SparkConf

#optionally configure spark
conf = (SparkConf().setAppName("V2Maestros").setMaster("local[2]").set("spark.executor.memory", "1g"))

#Initalize spark context onl runs once
sc = SparkContext(conf=conf)

In [ ]:
C:\spark\data\mllib\sample_kmeans_data.txt
C:\spark\data\mllib\sample_lda_libsvm_data.txt    
    

In [6]:
 autoData = sc.textFile(r'C:\Users\jeffnerd\Desktop\spark\auto-data.csv')

In [7]:
autoData.cache()

C:\Users\jeffnerd\Desktop\spark\auto-data.csv MapPartitionsRDD[1] at textFile at <unknown>:0

In [8]:
#remove the first line
first = autoData.first()
dataLines = autoData.filter(lambda x: x!=first)
dataLines.count()

197

In [9]:
import math

In [10]:
from pyspark.mllib.linalg import Vectors

In [21]:
from pyspark.ml.linalg import Vectors

In [22]:
#convert to alocal vector
def transformToNumeric(inputStr):
    attList = inputStr.split(",")
    
    doors = 1.0 if attList[3] == "two" else 2.0
    body = 1.0 if attList[4] == "sedan" else 2.0
    
    #Filterou columns not wanted at this stage
    values = Vectors.dense([doors, float(body), float(attList[7]),float(attList[8]),\
                            float(attList[9])])
    return values

In [23]:
autoVector = dataLines.map(transformToNumeric)
autoVector.persist()
autoVector.count()

197

In [ ]:
#centering and scaling this data i.e every value

In [13]:
#performng statistical analysis
from pyspark.mllib.stat import Statistics

In [14]:
autoStats = Statistics.colStats(autoVector)

In [15]:
colMeans = autoStats.mean()

In [16]:
colMeans

array([1.56852792e+00, 1.53299492e+00, 1.03604061e+02, 5.11802030e+03,
       2.51522843e+01])

In [17]:
colVariance = autoStats.variance()

In [18]:
colVariance

array([2.46555475e-01, 2.50181291e-01, 1.41670978e+03, 2.31395551e+05,
       4.14460789e+01])

In [ ]:
colstd = [np.sqrt(x) for x in colVariance]

In [19]:
import dill
colStdDev=dill.dumps(map(lambda x: math.sqrt(x), colVariance))

In [71]:
bcMeans = sc.broadcast(colMeans)

In [78]:
import pickle

In [80]:
bcVariance = sc.broadcast(colVariance)

In [20]:
#place the mean and sqrt to values in broadcast variable
bcMeans = sc.broadcast(colMeans)
bcVariance = sc.broadcast(colStdDev)

In [24]:
def centerAndScale(inVector):
    global bcMeans
    global bcVariance
    
    meanArray = bcMeans.value
    stdArray = bcVariance.value
    
    valueArray = inVector.toArray()
    retArray = []
    for i in range(valueArray.size):
        retArray.append( (valueArray[i] - meanArray[i]) /\
                        stdArray[i])
        return Vectors.dense(retArray)

In [25]:
csAuto = autoVector.map(centerAndScale)


In [26]:
csAuto.collect()

[DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([0.0034]),
 DenseVector([0.0034]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([0.0034]),
 DenseVector([-0.0044]),
 DenseVector([0.0034]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([0.0034]),
 DenseVector([0.0034]),
 DenseVector([0.0034]),
 DenseVector([0.0034]),
 DenseVector([-0.0044]),
 DenseVector([0.0034]),
 DenseVector([-0.0044]),
 DenseVector([0.0034]),
 DenseVector([0.0034]),
 DenseVector([0.0034]),
 DenseVector([-0.0044]),
 DenseVector([-0.0044]),
 DenseVector([0.0034]),
 DenseVector([-0.0044]),
 DenseVector([0.0034]),
 DenseVector([0.0034]),
 DenseVector([0.0034]),
 DenseVector([-0.0044]),
 DenseVector([0.

In [2]:
from pyspark.sql import DataFrame,SparkSession,Row

In [29]:
#create spark dataframe
autoRows = csAuto.map(lambda f: Row(features=f))
spark = SparkSession.builder.appName("kmeans1").master("local").config(conf=conf).getOrCreate()
autoDf = spark.createDataFrame(autoRows)

In [30]:
autoDf.select("features").show(10)

+--------------------+
|            features|
+--------------------+
|[-0.0044416243654...|
|[-0.0044416243654...|
|[-0.0044416243654...|
|[-0.0044416243654...|
|[-0.0044416243654...|
|[-0.0044416243654...|
|[-0.0044416243654...|
|[-0.0044416243654...|
|[-0.0044416243654...|
|[-0.0044416243654...|
+--------------------+
only showing top 10 rows



In [31]:
from pyspark.ml.clustering import KMeans

In [32]:
kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(autoDf)
predictions = model.transform(autoDf)

In [33]:
predictions.collect()

[Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([0.0034]), prediction=0),
 Row(features=DenseVector([0.0034]), prediction=0),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([-0.0044]), prediction=1),
 Row(features=DenseVector([0.0034]), prediction=0),
 Row(features=DenseVector([-0.0044]), prediction=

In [34]:
#plot the results in a scatter plot
import pandas as pd

In [35]:
def unstripData(inStr):
    return(inStr["prediction"],inStr["features"][0,inStr["features"][1],inStr["features"][2],\
                                                inStr["features"][3]])

In [37]:
unStripped = predictions.rdd.map(unstripData)
predList = unStripped.collect()
predPd = pd.DataFrame(predList)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 31, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 229, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 224, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-35-fc3dd40b3794>", line 2, in unstripData
  File "C:\spark\python\lib\pyspark.zip\pyspark\ml\linalg\__init__.py", line 399, in __getitem__
    return self.array[item]
IndexError: index 1 is out of bounds for axis 0 with size 1

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePyt

In [ ]:
import matplotlib.pylab as plt
plt.cla()
plt.scatter(predPd[3],predPd[4], c=predPd[0])


In [3]:
spark = SparkSession.builder.appName("kmins").master("local").config(conf=conf).getOrCreate()